In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 데이터 불러오기

In [2]:
# 데이터 생성
from sklearn.datasets import load_breast_cancer

def make_dataset():
    iris = load_breast_cancer()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['target'] = iris.target
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop('target', axis=1), df['target'], test_size=0.5, random_state=1004)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = make_dataset()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((284, 30), (285, 30), (284,), (285,))

test_size가 0.5이므로 5대5로 나누어진 것을 확인할 수 있다.

In [3]:
# target check
y_train.value_counts()

target
1    190
0     94
Name: count, dtype: int64

## 의사결정나무

In [6]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0) # 모델 선택

model.fit(X_train, y_train) # 모델 학습
pred = model.predict(X_test) # 모델 예측

accuracy_score(y_test,pred) # 정확도 평가

0.9263157894736842

### 의사결정나무 하이퍼파라미터
- criterion(default gini): 불순도 지표
- max_depth(default none): 최대 한도 깊이, 데이터 수가 많지 않을 경우 일정 깊이 이상은 같은 정확도 출력
- min_samples_split(default 2): 자식 노드를 갖기 위한 최소한의 데이터 수
- min_samples_leaf(default 1): 리프 노드가 되기 위한 최소 샘플 수
  

In [16]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion='entropy',
                               max_depth=7,
                               min_samples_split=2,
                               min_samples_leaf=2,
                               random_state=0) # 모델 선택, 하이퍼파라터

model.fit(X_train, y_train) # 모델 학습
pred = model.predict(X_test) # 모델 예측

accuracy_score(y_test,pred) # 정확도 평가

0.9228070175438596

## 랜덤포레스트
- 지도학습 알고리즘(분류, 회귀)
- 의사결정나무가 여러 개 있는 구성 = 의사결정나무의 앙상블
- 성능이 좋음(과대적합 가능성 낮음)
- 부트스트랩 샘플링(데이터셋 중복 허용)
- 최종 다수결 투표
- 앙상블 -> 배깅(랜덤포레스트), 부스팅(XGBoost)
  
*배깅: 같은 알고리즘으로 여러 모델 만들어 분류  
*부스팅: 학습과 예측하며 가중치 반

In [20]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9438596491228071

### 랜덤포레스트 하이퍼파라미터
- n_estimators (default 100) : 트리의 수
- criterion (default gini) : 불순도 지표
- max_depth (default None) : 최대 한도 깊이
- min_samples_split (default 2) : 자식 노드를 갖기 위한 최소한의 데이터 수
- min_samples_leaf (default 1) : 리프 노드가 되기 위한 최소 샘플 수

In [23]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, # 트리가 많으면 속도가 느리다.
                               max_depth=3,
                               random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9473684210526315

## XGBoost(eXtreme Gradient Boosting)

- 트리 앙상블 중 성능이 좋은 알고리즘
- 부스팅(앙상블) 기반의 알고리즘
- 약한 학습기가 계속해서 업데이트를 하며 좋은 모델을 만들어 감
- 캐글에서 뛰어난 성능을 보이면서 인기가 높아짐

In [26]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

# warning ignore
# 1) use_label_encoder=False
# 2) eval_metric='logloss'

0.9614035087719298

## XGBoost 하이퍼파라미터
- booster (default gbtree) : 부스팅 알고리즘 (또는 dart, gblinear)
- objective (default binary:logistic) : 이진분류 (다중분류: multi:softmax)
- max_depth (default 6) : 최대 한도 깊이
- learning_rate (default 0.1) : 학습률  
  경사하강법: 기울기가 0인 지점을 찾아나감
- n_estimators (default 100) : 트리의 수  
  learning_rate를 낮췄다면 n_estimators는 올려주어야 한다
- subsample (default 1) : 훈련 샘플 개수의 비율
- colsample_bytree (default 1) : 특성 개수의 비율
- n_jobs (default 1) : 사용 코어 수 (-1: 모든 코어를 다 사용)

In [46]:
model = XGBClassifier(random_state=0,
                     booster='gbtree',
                     objective='binary:logistic',
                     max_depth=5,
                     learning_rate=0.1,
                     n_estimators=500, # 500이 되기 전에 괜찮은 점수가 나오면 조기종료
                     subsample=1,
                     colsample_bytree=1,
                     n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

# learning_rate=0.1, n_estimators=200 -> 0.9649122807017544
# learning_rate=0.05, n_estimators=500 -> 0.9614035087719298
# learning_rate=0.1, n_estimators=500 -> 0.9649122807017544

0.9649122807017544

### 조기종료

In [63]:
model = XGBClassifier(random_state=0,
                      use_label_encoder=False,
                      eval_metric='logloss', 
                     learning_rate = 0.1,
                      n_estimators = 500, # 500이 되기 전에 괜찮은 점수가 나오면 조기종료)
                      early_stopping_rounds=10) # 10번 이상 돌려도 성능 향상이 없으면 조기종료
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

[0]	validation_0-logloss:0.61495
[1]	validation_0-logloss:0.55350
[2]	validation_0-logloss:0.50426
[3]	validation_0-logloss:0.45990
[4]	validation_0-logloss:0.42274
[5]	validation_0-logloss:0.39294
[6]	validation_0-logloss:0.36399
[7]	validation_0-logloss:0.33934
[8]	validation_0-logloss:0.32124
[9]	validation_0-logloss:0.30207
[10]	validation_0-logloss:0.28493
[11]	validation_0-logloss:0.27252
[12]	validation_0-logloss:0.25879
[13]	validation_0-logloss:0.24627
[14]	validation_0-logloss:0.23617
[15]	validation_0-logloss:0.22768
[16]	validation_0-logloss:0.21963
[17]	validation_0-logloss:0.21348
[18]	validation_0-logloss:0.20831
[19]	validation_0-logloss:0.20246
[20]	validation_0-logloss:0.19794
[21]	validation_0-logloss:0.19437
[22]	validation_0-logloss:0.19148
[23]	validation_0-logloss:0.18880
[24]	validation_0-logloss:0.18508
[25]	validation_0-logloss:0.18194
[26]	validation_0-logloss:0.18122
[27]	validation_0-logloss:0.17886
[28]	validation_0-logloss:0.17772
[29]	validation_0-loglos

C:\Users\juwonhee\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [12:31:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


[55]	validation_0-logloss:0.16983
[56]	validation_0-logloss:0.17076
[57]	validation_0-logloss:0.17019


0.9508771929824561

## 교차검증